In [ ]:
pip install gradio

In [ ]:
pip install sdmetrics

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sdmetrics.single_column import KSComplement, CategoryCoverage, TVComplement

In [ ]:
from pandas.api.types import is_numeric_dtype
from sdmetrics.reports.single_table import QualityReport
from sdv.evaluation.single_table import get_column_pair_plot

In [ ]:
from sdv.metadata import SingleTableMetadata
from sdmetrics.visualization import get_column_plot
metadata = SingleTableMetadata()

In [ ]:
train = pd.read_csv('/content/sample_data/california_housing_train.csv')
test = pd.read_csv('/content/sample_data/california_housing_test.csv')
one = pd.read_csv('/content/sample_data/california_housing_test.csv')
two = pd.read_csv('/content/sample_data/california_housing_test.csv')
three = pd.read_csv('/content/sample_data/california_housing_test.csv')

column = list(train.columns)

In [ ]:
def plot(df, col):

  fig = px.histogram(df, x=col)
  fig.show()

  return fig

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train)
metadata

{
    "columns": {
        "longitude": {
            "sdtype": "longitude",
            "pii": true
        },
        "latitude": {
            "sdtype": "latitude",
            "pii": true
        },
        "housing_median_age": {
            "sdtype": "numerical"
        },
        "total_rooms": {
            "sdtype": "numerical"
        },
        "total_bedrooms": {
            "sdtype": "numerical"
        },
        "population": {
            "sdtype": "numerical"
        },
        "households": {
            "sdtype": "numerical"
        },
        "median_income": {
            "sdtype": "numerical"
        },
        "median_house_value": {
            "sdtype": "numerical"
        },
        "keys": {
            "sdtype": "categorical"
        }
    },
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

In [ ]:
metadata =  metadata.to_dict()
metadata

{'columns': {'longitude': {'sdtype': 'longitude', 'pii': True},
  'latitude': {'sdtype': 'latitude', 'pii': True},
  'housing_median_age': {'sdtype': 'numerical'},
  'total_rooms': {'sdtype': 'numerical'},
  'total_bedrooms': {'sdtype': 'numerical'},
  'population': {'sdtype': 'numerical'},
  'households': {'sdtype': 'numerical'},
  'median_income': {'sdtype': 'numerical'},
  'median_house_value': {'sdtype': 'numerical'},
  'keys': {'sdtype': 'categorical'}},
 'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1'}

In [ ]:
import plotly.figure_factory as ff
import numpy as np
np.random.seed(1)


def distplot(df, col):
  fig = ff.create_distplot([df[col]], [col])

  return fig

def big_distplot(df,df2, col,col2):
  fig = ff.create_distplot([df[col],df2[col2]], [col,col2])

  return fig

def histogram_score(df,df2, col,col2):
  ks = KSComplement()
  real_table = df[col]
  synthetic_table = df2[col2]
  score = ks.compute(
      real_data=df[col],
      synthetic_data=df2[col2]
  )
  return score

def combined_df(df,df2):
  df['keys'] ='train'
  df2['keys'] ='test'
  df = pd.concat([df,df2])
  return df
def scatter_plot(df, col1, col2):
  fig = px.scatter(df, x=col1, y=col2, color="keys", trendline="ols")

  return fig

def change(df, col):
  comp = []
  print(df.columns)
  for i in df.columns:
    if col != i and is_numeric_dtype(df[col]) and is_numeric_dtype(df[i]):
        plot = gr.Plot(value = scatter_plot(df,col,i))
        comp.append(plot)
  return comp

def get_value(value):
  return value


def create():

  with gr.Column():
      for label in range(1,2):
          gr.TextArea(label=f"Samples for class: {label} vs {check.value}")
          gr.Plot(combined,label,check.value)


In [ ]:
from sdmetrics.single_column.base import SingleColumnMetric

In [ ]:
x = real_ratio(train, test, 'households','households')
x

0.5557471264367816

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
      with gr.Column(variant ='panel'):
          check = gr.Radio(choices = column, label="Company Selection",value =  column[0])
      with gr.Column(variant ='panel'):
          drop =gr.Dropdown(range(1,100), label="Select the experiment to distplot", value = 1)
          btn = gr.Button(value="Update Filter")

    with gr.Row():
      with gr.Column(variant ='panel'):
          df1 = gr.DataFrame(label = 'Orginal Data', value = train)
          output = gr.Plot()

      with gr.Column(variant ='panel'):
          df2 = gr.DataFrame(label = 'Orginal Data', value = test)
          output2 = gr.Plot()

    with gr.Row():
      big_plot = gr.Plot()

    with gr.Row():
      result = gr.Textbox(label="KS Component score ")


    combined = gr.DataFrame(label = 'Orginal Data', render = True, value = combined_df(train, test))

    comp = []
    for i in column:
      if check.value != i:
        plot = gr.Plot()
        comp.append(plot)



    btn.click(fn=distplot, inputs=[df1, check], outputs=[output])
    btn.click(fn=distplot, inputs=[df2, check], outputs=[output2])
    btn.click(fn=big_distplot, inputs=[df1, df2, check,check], outputs=[big_plot])
    btn.click(fn=histogram_score, inputs=[df1, df2, check,check], outputs=[result])
    btn.click(fn=combined_df, inputs=[df1, df2], outputs=[combined])

    btn.click(fn=change, inputs=[combined,check] ,outputs = comp )

    demo.load(fn=distplot, inputs=[df1, check], outputs=[output])
    demo.load(fn=distplot, inputs=[df2, check], outputs=[output2])
    demo.load(fn=big_distplot, inputs=[df1, df2, check,check], outputs=[big_plot])
    demo.load(fn=histogram_score, inputs=[df1, df2, check,check], outputs=[result])
    demo.load(fn=combined_df, inputs=[df1, df2], outputs=[combined])
    demo.load(fn=change, inputs=[combined,check] ,outputs = comp )

demo.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c5049019120c8145fd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'keys'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'keys'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'keys'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'keys'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'keys'],
      dtype='object')


In [ ]:
plot(train, 'housing_median_age')

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')


AttributeError: 'Figure' object has no attribute 'plot'

<Figure size 640x480 with 0 Axes>

In [ ]:
train.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

In [ ]:
df = px.data.tips()

In [ ]:
train['keys'] ='train'
test['keys'] ='test'
t = pd.concat([train,test])
t

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,keys
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0,train
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0,train
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0,train
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0,train
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0,train
...,...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0,test
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0,test
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0,test
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0,test


In [ ]:
fig = px.scatter(t, x="longitude", y="latitude", color="keys", trendline="ols")
fig.show()

In [ ]:
fig = px.scatter(df, x="total_bill", y="tip", facet_col="smoker", color="sex", trendline="ols")
fig.show()

In [ ]:
def words():
    sentence = "A test of Gradio"
    words = sentence.split()
    update_show = [gr.Button(visible=True, value=w) for w in words]
    update_hide = [gr.Button(visible=False, value="") for _ in range(10-len(words))]
    return update_show + update_hide

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("# Train a Text Classifier with Synthetic Data")
    labels = gr.Dropdown(choices=[], value=[], label="Classes", allow_custom_value=True, multiselect=True)

    @gr.render(inputs=[labels])
    def show_textbox(labels_):
        with gr.Row():
            for label in labels_:
                gr.TextArea(label=f"Samples for class: {label}")
        if len(labels_)>=2:
            with gr.Row():
                gr.Button("TRAIN!", variant="primary")

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a7e9dbcd877cb73b3d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
